In [ ]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector,ParameterExpression
from qiskit.primitives import Estimator,Sampler,BackendEstimator,BackendSampler
from qiskit.quantum_info import SparsePauliOp
from qiskit import transpile
from qiskit import Aer


from squlearn.feature_map import ChebPQC
from squlearn.expectation_operator import SinglePauli,SummedPaulis
from squlearn.qnn.qnn import QNN
from squlearn.util import Executor

import copy

#from circuit_tree import *

from squlearn.util.optree import *
from squlearn.util.optree.optree import *
from squlearn.util.optree.optree_evaluate import *

num_qubits = 10

pqc = ChebPQC(num_qubits,1,3)

x = ParameterVector("x",pqc.num_features)
p = ParameterVector("p",pqc.num_parameters)

QC = pqc.get_circuit(x,p)



SUPPORTED_GATES = {"x", "y", "z", "h", "rx", "ry", "rz", "p", "u", "cx", "cy", "cz"} #rzz,ryy,rxx also work!
QC = transpile(QC,basis_gates=SUPPORTED_GATES)
QC.draw()

In [ ]:
d = derivative(OpTreeLeafCircuit(QC),p)

In [ ]:
grad = simplify_copy(d)
#grad = d

In [ ]:
len(str(grad))

In [ ]:
xval = [-0.5]
np.random.seed(0)
pval = np.random.rand(pqc.num_parameters)
dictionary = dict(zip(p, pval))
dictionary.update(dict(zip(x, xval)))


In [ ]:
from pympler import asizeof
circ = get_first_leaf(grad)
full_size = asizeof.asizeof(circ)
print(f"Size of circuit: {full_size} bytes")


In [ ]:
op = SummedPaulis(num_qubits)
pop = ParameterVector('p',op.num_parameters)
op_dict=dict(zip(pop,np.random.rand(op.num_parameters)))
pauli_op = op.get_pauli_new(pop)

In [ ]:
evaluate_estimator(grad,pauli_op,dictionary,op_dict,Estimator(),detect_circuit_duplicates=False)

In [ ]:
evaluate_sampler(grad,pauli_op,dictionary,op_dict,Sampler(),detect_circuit_duplicates=False)

In [ ]:
estimator_qasm = BackendEstimator(Aer.get_backend('qasm_simulator'),options={'shots':10000})
sampler_qasm = BackendSampler(Aer.get_backend('qasm_simulator'),options={'shots':10000})

In [ ]:
evaluate_estimator(grad,pauli_op,dictionary,op_dict,estimator_qasm,detect_circuit_duplicates=False)

In [ ]:
evaluate_estimator(grad,pauli_op,dictionary,op_dict,estimator_qasm,detect_circuit_duplicates=True)

In [ ]:
# evaluate_sampler(grad,pauli_op,dictionary,op_dict,sampler_qasm,detect_circuit_duplicates=False)

In [ ]:
total_dic = dictionary
total_dic.update(op_dict)

In [ ]:
expec_tree = gen_expectation_tree(grad,pauli_op)

In [ ]:
total_dic2 = total_dic.copy()
total_dic2[pop[0]]=-1
total_dic2[pop[1]]=2
total_dic2[x[0]]=-0.2
evaluate_expectation_tree_from_estimator(expec_tree,total_dic2,Estimator())

In [ ]:
evaluate_expectation_tree_from_estimator(expec_tree,[total_dic,total_dic2],Estimator())

In [ ]:
expec_tree = gen_expectation_tree(grad,pauli_op)

In [ ]:
evaluate_expectation_tree_from_sampler(expec_tree,total_dic,Sampler())